In [1]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [7]:
df = pd.read_csv('./musinsa_review_skirt.csv')

In [12]:
df = df.sort_values('reviewImg')

In [ ]:
df.head(50)

In [4]:
df.shape

(16229, 19)

In [5]:
len(df['userName'].unique())

8606

In [14]:
df_1 = df.drop_duplicates(['userName', 'date', 'goodsNo']).reset_index(drop=True)

In [18]:
df_1.shape

(12501, 19)

In [20]:
len(df_1['userName'].unique())

8606

In [2]:
df_one = pd.read_csv('./musinsa_review_onepiece.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb4 in position 228: invalid start byte

In [ ]:
df_one = df_one.sort_values('reviewImg')

In [ ]:
df_one.head(50)

In [4]:
df_one.shape

(16229, 19)

In [5]:
len(df_one['userName'].unique())

8606

In [14]:
df_one1 = df_one.drop_duplicates(['userName', 'date', 'goodsNo']).reset_index(drop=True)

In [18]:
df_one1.shape

(12501, 19)

In [20]:
len(df_one1['userName'].unique())

8606